### <center><font color=navy> Tutorial #5 Computer- and robot-assisted surgery</font></center>
## <center><font color=navy> Computer Vision Basics II</font></center>
<center>&copy; Sebastian Bodenstedt, National Center for Tumor Diseases (NCT) Dresden<br>
    <a href="https://www.nct-dresden.de/"><img src="https://www.nct-dresden.de/++theme++nct/images/logo-nct-en.svg"></a> </center>

## <center><font color=navy>Preperation</font></center>

For this tutorial, we will utilize the OpenCV, Matplotlib and NumPy:

In [ ]:
import cv2
import numpy as np
# Force Matplotlib to display data directly in Jupyter
%matplotlib inline 
from matplotlib import pyplot as plt

We will also download and extract a few image sequences:

In [ ]:
import urllib.request
from os.path import basename, exists
import zipfile

def download_and_extract(url): #download and extract Zip archive
    file_path = basename(url)
    if not exists(file_path): # does zip file already exist?
        urllib.request.urlretrieve(url, file_path) # if not, download it
        with zipfile.ZipFile(file_path, 'r') as zip_ref: # and unzip it
            zip_ref.extractall(".")

In [ ]:
#download_and_extract("http://tso.ukdd.de/crs/Exercise1.zip") # In case you didn't download the data last week
download_and_extract("http://tso.ukdd.de/crs/CVBasicsII_chessboard.zip")
download_and_extract("http://tso.ukdd.de/crs/CVBasicsII_sequence1.zip")
download_and_extract("http://tso.ukdd.de/crs/CVBasicsII_sequence2.zip")

We now list the extracted files:

In [ ]:
!dir *

## <center><font color=navy>Review</font></center>
We can utilize OpenCV to read one of the images from HD and NumPy to process the data. And Numpy can be used for visualization:

In [ ]:
img = cv2.imread("Exercise1/img_01_raw.png") # Read image from HD

img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert BGR to RGB

plt.imshow(img_rgb) # Display result

Images can also be converted to grayscale and displayed:

In [ ]:
def show_gray(img, canvas=plt, title=""): # Later we want to draw on a different underground, so we define this as a parameter
    canvas.imshow(img, cmap='gray', vmin=0, vmax=255)
    if not title == "":
        canvas.set_title(title)

In [ ]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convert BGR to grayscale

show_gray(img_gray,) # Display results

## <center><font color=navy>Filtering with OpenCV</font></center>
Attempt to implement your own convolutional filter for grayscale images, for a custom kernel, e.g. Box, Prewitt, Sobel, Laplace, ...

In [ ]:
kernel_box = np.ones((3,3), dtype=np.float32)
kernel_box/=9

print(kernel_box)


Or Gaussian filter:

In [ ]:
def gaussian_filter(r, muu=0, sigma=1):
    x, y = np.meshgrid(np.linspace(-1, 1, 2*r+1),
                       np.linspace(-1, 1, 2*r+1))
    dst = np.sqrt(x**2+y**2)
 
    normal = 1/(2 * np.pi * sigma**2)
    gauss = np.exp(-((dst-muu)**2 / (2.0 * sigma**2))) * normal
    
    gauss/= np.sum(gauss)
    
    return gauss

In [ ]:
kernel_gaussian = gaussian_filter(2, 0, 0.625)
print(kernel_gaussian)

In [ ]:
def apply_kernel(image, kernel):
    assert kernel.shape[0] == kernel.shape[1]
    assert kernel.shape[0] % 2 == 1
    
    r = (kernel.shape[0] - 1)//2
    
    #TODO Apply filter
    
    return image

In [ ]:
result = apply_kernel(img_gray, kernel_prewitt_x)
print(np.min(result), np.max(result), result.dtype)

figure, axis = plt.subplots(1, 2, figsize=(15, 15)) # subplots let you visualize multiple outputs simultanously

show_gray(img_gray, axis[0])
show_gray(result, axis[1])

OpenCV has implement its own convolutional filtering method (higher performance).

In [ ]:
figure, axis = plt.subplots(1, 2, figsize=(15, 15)) # subplots let you visualize multiple outputs simultanously

result_cv = cv2.filter2D(img_gray, kernel=kernel_prewitt_x, ddepth=-1)

show_gray(img_gray, axis[0])
show_gray(result_cv, axis[1])

The OpenCV also contains implementations for the Median Filter and the Bilateral Filter:

In [ ]:
figure, axis = plt.subplots(1, 2, figsize=(15, 15)) # subplots let you visualize multiple outputs simultanously

result_cv = cv2.medianBlur(img_gray, 31) # Median Filter over 3x3 neighborhood

show_gray(img_gray, axis[0])
show_gray(result_cv, axis[1])

In [ ]:
figure, axis = plt.subplots(1, 2, figsize=(15, 15)) # subplots let you visualize multiple outputs simultanously

result_cv = cv2.bilateralFilter(img_rgb, d=-1, sigmaColor=51, sigmaSpace=51)

show_gray(img_gray, axis[0])
show_gray(result_cv, axis[1])

Implement a few other filter matrices and try out the different filters and combinations:

In [ ]:
figure, axis = plt.subplots(1, 2, figsize=(30, 30)) # subplots let you visualize multiple outputs simultanously

show_gray(img_gray, axis[0, 0], "Original")
result_cv = cv2.filter2D(img_gray, kernel=kernel_box, ddepth=-1)
show_gray(result_cv, axis[0, 1], "Box Filter")
result_cv = cv2.filter2D(img_gray, kernel=kernel_gaussian, ddepth=-1)

# TODO: Try out different filters and combinations


## <center><font color=navy>Stereo-camera calibration with OpenCV</font></center>
OpenCV also provides functionality for loading image sequences and videos:

In [ ]:
cap = cv2.VideoCapture("CVBasicsII/chessboard/scene_left%04d.png") # Load image sequence

done = False

while not done: # Iterate over the sequence until finished
    ret, frame = cap.read()
    done = not ret
    
    if ret:
        cv2.imshow("Video", frame) # Display frame
        cv2.waitKey(100)
    
cv2.destroyAllWindows() # Very important! Otherwise, Jupyter will hang

Traditionally images of chessboard patterns are used for calibration. OpenCV contains functionality for detecting and drawing chessboard patterns. First we need to define the size of the pattern. In our case we have 17 columns and 12 rows, with 5x5 mm squares:

In [ ]:
cols = 17
rows = 12
sqrsize = 5

flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE + cv2.CALIB_CB_FAST_CHECK

We can then detected the pattern in images using OpenCV and draw them:

In [ ]:
img = cv2.imread("CVBasicsII/chessboard/scene_left0000.png")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # The image has to be converted to grayscale

retval, corners = cv2.findChessboardCorners(img_gray, (cols-1, rows-1), flags=flags)

img_corners = cv2.drawChessboardCorners(img, (cols-1,rows-1), corners, retval)

plt.imshow(img_corners)

We can refine these to subpixel level:

In [ ]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

corners_sub = cv2.cornerSubPix(img_gray, corners, (5, 5), (-1, -1), criteria)

img_corners = cv2.drawChessboardCorners(img, (cols-1,rows-1), corners_sub, retval)

plt.imshow(img_corners)

We can now combine the locating of the corner pixels with reading image sequences, to locate chessboard pattern in the left (scene_left%04d.png) and right (scene_right%04d.png) sequences and save them:

In [ ]:
cap_left = cv2.VideoCapture("CVBasicsII/chessboard/scene_left%04d.png") # Load image sequence left
cap_right = cv2.VideoCapture("CVBasicsII/chessboard/scene_right%04d.png") # Load image sequence right

corners_left = [] # Save corners
corners_right = []
width = 0
height = 0

done = False

while not done: # Iterate over the sequence until finished
    # TODO Iterate through all images and save patterns if corresponding left and right images both contain one at the current time point. Also determine width and height of image

Let's check how many patterns we found in both images:

In [ ]:
print(len(all_corners_left), len(all_corners_right))


To calibrate the stereo camera system, we need to define the 3D-geometry of the pattern:

In [ ]:
objp = np.zeros((1, (rows-1)*(cols-1), 3), np.float32)
objp[0,:,:2] = np.mgrid[0:cols-1, 0:rows-1].T.reshape(-1, 2)*sqrsize
print(objp)

Furthermore, we need to match a 3D-pattern to each located 2D-Pattern:

In [ ]:
objpoints = []
for i in range(len(all_corners_left)):
    objpoints.append(objp) # Each image shows the same pattern (just in different poses), we can therefore use the same 3D pattern

Now we are already ready to calibrate our stereo camera system:

In [ ]:
stereocalibration_flags = cv2.CALIB_FIX_K3
stereocalibration_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 50, 0.000001)

ret, K1, d1, K2, d2, R, T, E, F = cv2.stereoCalibrate(objpoints, all_corners_left, all_corners_right, None, None, None, None, (width, height), criteria=stereocalibration_criteria, flags=stereocalibration_flags)

We should now have computed sensible values for the camera matrices, the distortions and the extrinsic parameters:

In [ ]:
print(K1)
print(K2)
print(d1)
print(d2)
print(R)
print(T)

## <center><font color=navy>Stereo rectification</font></center>
For stereo triangulation, we need to locate matching points in the two images. Here the epipolar geometry can help. Let's say, we have a point in the left image:

In [ ]:
img_left = cv2.imread("CVBasicsII/sequence1/scene_left0000.png")
img_right = cv2.imread("CVBasicsII/sequence1/scene_right0000.png")

px = 110
py = 208
img_left[py-3:py+3,px-3:px+3, :] = 255

figure, axis = plt.subplots(1, 2, figsize=(30, 30)) # subplots let you visualize multiple outputs simultanously

axis[0].imshow(img_left[:, :, ::-1])
axis[1].imshow(img_right[:, :, ::-1])

We can narrow the search space using epipolar geometry:

In [ ]:
p_l = np.asarray([px, py, 1])
l_r = F@p_l # Returns an epipolar line in the right image
print(l_r)
a, b, c = l_r.ravel() # Get the 3 line parameters from the line

x = np.array([0, img_right.shape[1]]) # Calculate y-coordinates for 0 and image width
y = -(x*a + c) / b
print(x, y)
y = y.astype(np.int64)
cv2.line(img_right, (x[0], y[0]), (x[1], y[1]), (255, 255, 255), thickness=2) # Draw line into the image

figure, axis = plt.subplots(1, 2, figsize=(30, 30)) # subplots let you visualize multiple outputs simultanously

axis[0].imshow(img_left[:, :, ::-1])
axis[1].imshow(img_right[:, :, ::-1])

Rectification can make locating matches faster:

In [ ]:
R1, R2, P1, P2, Q, roi_left, roi_right = cv2.stereoRectify(K1, d1, K2, d2, (width, height), R, T, flags=cv2.CALIB_ZERO_DISPARITY, alpha=0.9)

We can use these parameters to directly rectify images:

In [ ]:
map1x, map1y = cv2.initUndistortRectifyMap(K1, d1, R1, P1, (width, height), cv2.CV_32FC1)
map2x, map2y = cv2.initUndistortRectifyMap(K2, d2, R2, P2, (width, height), cv2.CV_32FC1)

In [ ]:
left_rec = cv2.remap(img_left, map1x, map1y, cv2.INTER_LINEAR)
right_rec = cv2.remap(img_right, map2x, map2y, cv2.INTER_LINEAR)

figure, axis = plt.subplots(1, 2, figsize=(30, 30)) # subplots let you visualize multiple outputs simultanously

axis[0].imshow(left_rec[:, :, ::-1])
axis[1].imshow(right_rec[:, :, ::-1])

### <center><font color=navy>3D reconstruction</font></center>
We can use the rectified images now as input for the semi-global block matching:

In [ ]:
img_left = cv2.imread("CVBasicsII/sequence1/scene_left0015.png")
img_right = cv2.imread("CVBasicsII/sequence1/scene_right0015.png")

left_rec = cv2.remap(img_left, map1x, map1y, cv2.INTER_LINEAR)
right_rec = cv2.remap(img_right, map2x, map2y, cv2.INTER_LINEAR)

minDisparity = 10
maxDisparity = 192
blocksize = 3

p1 = 8*3*blocksize*blocksize
p2 = 32*3*blocksize*blocksize

stereoProcessor = cv2.StereoSGBM_create(minDisparity, maxDisparity, blocksize, p1, p2)

disp = stereoProcessor.compute(left_rec, right_rec)

figure, axis = plt.subplots(1, 2, figsize=(30, 30)) # subplots let you visualize multiple outputs simultanously

axis[0].imshow(left_rec[:, :, ::-1])
axis[1].imshow(disp)


In [ ]:
#TODO Visualize/reconstruct entire video